In [30]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision.transforms.v2 import PILToTensor, ToDtype
from pathlib import Path
import matplotlib.pyplot as plt
import os

In [31]:
data_path = Path("data")
image_path = data_path / "CIFAR10"
image_transform = torchvision.transforms.Compose([PILToTensor(),
                                                 ToDtype(torch.float32)])
train_data = torchvision.datasets.CIFAR10(root=image_path,
                                         train=True,
                                         download=True,
                                         transform=image_transform)
test_data = torchvision.datasets.CIFAR10(root=image_path,
                                         train=False,
                                         download=True,
                                        transform=image_transform)
print(train_data, test_data)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data\CIFAR10
    Split: Train
    StandardTransform
Transform: Compose(
               PILToTensor()
               ToDtype(scale=False)
           ) Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data\CIFAR10
    Split: Test
    StandardTransform
Transform: Compose(
               PILToTensor()
               ToDtype(scale=False)
           )


In [32]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_WORKERS)
print(train_dataloader, test_dataloader)
print(f"Train dataloader lenght: {len(train_dataloader)}\nTest dataloader lenght: {len(test_dataloader)}")

<torch.utils.data.dataloader.DataLoader object at 0x000002180A330610> <torch.utils.data.dataloader.DataLoader object at 0x000002180A330BB0>
Train dataloader lenght: 1563
Test dataloader lenght: 313


In [33]:
image_batch, label_batch = next(iter(train_dataloader))
image_batch.shape

torch.Size([32, 3, 32, 32])

In [34]:
train_data.classes, image_batch.dtype

(['airplane',
  'automobile',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'],
 torch.float32)

In [51]:
class TinyVGG(nn.Module):
    def __init__(self, input_size: int, hidden_units: int, output_size: int):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(nn.Conv2d(in_channels=input_size,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    padding=0,
                                                    stride=1),
                                          nn.ReLU(),
                                          nn.Conv2d(in_channels=hidden_units,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    padding=0,
                                                    stride=1),
                                          nn.ReLU(),
                                          nn.MaxPool2d(kernel_size=2,
                                                       stride=1))
        self.conv_layer_2 = nn.Sequential(nn.Conv2d(in_channels=hidden_units,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    padding=0,
                                                    stride=1),
                                          nn.ReLU(),
                                          nn.Conv2d(in_channels=hidden_units,
                                                    out_channels=hidden_units,
                                                    kernel_size=3,
                                                    padding=0,
                                                    stride=1),
                                          nn.ReLU(),
                                          nn.MaxPool2d(kernel_size=2,
                                                       stride=1))
        self.linear_layer = nn.Sequential(nn.Flatten(),
                                          nn.Linear(in_features=hidden_units*22*22,
                                                    out_features=output_size))
    def forward(self, x: torch.Tensor):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.linear_layer(x)
        return x

In [52]:
model1 = TinyVGG(input_size = 3, hidden_units = 10, output_size = len(train_data.classes))
model1

TinyVGG(
  (conv_layer_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4840, out_features=10, bias=True)
  )
)

In [53]:
model1(image_batch)

tensor([[ 3.9263e-02, -5.7457e-01, -3.8242e-02,  1.8610e-01, -2.0569e-01,
         -3.6785e-01, -1.3927e-01, -2.5597e-01,  1.6878e-01,  5.2950e-01],
        [-2.5418e-01, -2.2228e-01, -1.9802e-01,  2.6527e-01, -5.2416e-01,
          8.9739e-02, -2.7519e-01,  7.6876e-01,  1.5915e-01,  1.7891e-01],
        [-3.8404e-01, -2.7174e-01,  2.5367e-01,  2.0262e-01, -8.8375e-01,
         -6.4694e-01,  1.2264e-01, -1.5233e-01, -1.2184e-01,  1.0719e+00],
        [-1.4112e-01, -5.0311e-01, -5.8862e-01,  1.2111e-01, -1.3987e-01,
         -3.5381e-01, -4.1576e-01, -6.7284e-01,  3.0174e-01,  4.2602e-01],
        [-5.9583e-01, -8.2177e-01, -4.5160e-01,  4.3052e-01, -4.9938e-01,
         -9.0416e-01, -7.8124e-01, -8.7375e-02,  1.0801e-01,  2.7711e-01],
        [-6.2233e-01, -1.0075e+00, -4.1872e-01,  2.4204e-01, -7.8757e-01,
         -6.6359e-01, -5.8333e-01, -3.9064e-01,  2.6214e-01,  6.0758e-01],
        [-4.5958e-02, -1.9159e-01, -2.8046e-01, -9.4460e-02, -6.5996e-02,
         -3.7428e-01, -2.7937e-0

In [55]:
def train_pass(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer) -> torch.Tensor:
    model.train()
    train_loss = 0
    for batch,(X,y) in enumerate(dataloader):
        y_preds = model(X)
        loss = loss_fn(y_preds,y)
        train_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss = train_loss/len(dataloader)
    return train_loss

In [56]:
train_pass(model1, [(image_batch, label_batch)], nn.CrossEntropyLoss(), torch.optim.SGD(params=model1.parameters(), lr=0.1))

tensor(17.5978, grad_fn=<DivBackward0>)